## Train Multiple Classifiers

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
from sklearn.metrics import accuracy_score
import random
import numpy as np
import torch
import os

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set Random Seed for Reproducibility

In [2]:
# Set a seed for random module
random.seed(42)

# Set a seed for numpy module
np.random.seed(42)

# Set a seed for torch module
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Define Parameters

In [3]:
TOPICS = ["cannabis", "kinder", "energie"]
MODEL = "deepset/gelectra-large"#"deepset/gelectra-large"
STRATEGIES = ["random", "stratified" , "clustered"]#, "shared_domain"]
SUFFIX = "" #"", "_holdout", "_extended"
MAX_CONTENT_LENGTH = 384 # 496, 192, 384
OVERLAP = 64
FEATURES = "url_and_content" # "url", "content", "url_and_content"

In [4]:
from torch import nn

In [5]:
# Define training arguments
TRAINING_ARGS = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,             # number of training epochs # TODO
    weight_decay=0.01,  # Weight decay if we apply some form of weight regularization.
    logging_dir='./logs',  # Directory where the training logs will be stored.
    logging_strategy="steps",  # The logging strategy determines when to log
    logging_steps=100,  # Number of steps between logging of training loss.
    evaluation_strategy="steps",  # Evaluation is done
    eval_steps=100,  # Number of steps between evaluations.
    load_best_model_at_end=True,  # load the best model at the end of training.
    metric_for_best_model="eval_loss",
    lr_scheduler_type='linear',  # The scheduler type to use, e.g., 'linear', 'cosine'
    warmup_ratio=0.1  # Proportion of training to perform linear learning rate warmup for.
)

**Extract URL-path:**

In [6]:
from urllib.parse import urlparse, urlunparse

def extract_url_path(example):
    view_url = example['view_url']
    if "://" not in view_url:
        view_url = "http://" + view_url  # Assume http if no protocol specified
    parsed_url = urlparse(view_url)
    new_url = urlunparse(('', '', parsed_url.path, parsed_url.params, parsed_url.query, parsed_url.fragment))
    example['url_path'] = new_url.lstrip('/')  # Store the result in a new field
    return example


extract_url_path({"view_url": "https://www.google.com/search?q=python+url+path"})

{'view_url': 'https://www.google.com/search?q=python+url+path',
 'url_path': 'search?q=python+url+path'}

## Train Models

In [7]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

In [8]:
for topic in TOPICS: # ----------------------------------------------------------------------
    for strategie in STRATEGIES: # -------------------------------------------------------------
        print(f"Training on {topic} dataset using {strategie} strategy")
        dataset = load_from_disk(f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{strategie}{SUFFIX}_{MAX_CONTENT_LENGTH}")

Training on cannabis dataset using random strategy
Training on cannabis dataset using stratified strategy
Training on cannabis dataset using clustered strategy
Training on kinder dataset using random strategy
Training on kinder dataset using stratified strategy
Training on kinder dataset using clustered strategy
Training on energie dataset using random strategy
Training on energie dataset using stratified strategy
Training on energie dataset using clustered strategy


In [9]:
from collections import defaultdict
from transformers import EarlyStoppingCallback

training_results = defaultdict(dict)

for topic in TOPICS: # ----------------------------------------------------------------------
    
    for strategie in STRATEGIES: # -------------------------------------------------------------

        print(f"Training on {topic} dataset using {strategie} strategy")
        
        if FEATURES == "url":
            dataset = load_from_disk(
                f"../../data/tmp/processed_dataset_{topic}_buffed_{strategie}")
            # Extract the path from the URL
            dataset = dataset.map(extract_url_path)
            # dataset['test'] = sample_random_from_dataset(dataset, n=5, subset='test')
        else:
            dataset = load_from_disk(
                f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{strategie}{SUFFIX}_{MAX_CONTENT_LENGTH}")
            # Extract the path from the URL
            dataset = dataset.map(extract_url_path)
            # dataset['test'] = sample_random_from_dataset(dataset, n=5, subset='test')
        
        # Load model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

        # Define tokenization strategies
        tokenization_strategies = {
            
            # Tokenize the content of the page
            "content": lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True),
            
            # Tokenize the URL path
            "url": lambda examples: tokenizer(examples["url_path"], padding="max_length", truncation=True),
            
            # Tokenize the URL path and the content of the page
            "url_and_content": lambda examples: tokenizer(examples["url_path"], examples["text"], padding="max_length", truncation=True)
        }

        # Tokenize dataset
        tokenized_datasets = dataset.map(tokenization_strategies[FEATURES],
            batched=True)
        
        # Create a Trainer object
        trainer = Trainer(
            model=model,
            args=TRAINING_ARGS,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            compute_metrics=compute_metrics,
            #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )
        
        # Train the model
        training_result = trainer.train()
        
        # Update the eval_results dictionary
        training_results[strategie][topic] = training_result
        
        # Evaluate the model
        eval_results = trainer.evaluate(tokenized_datasets["test"])
        print("Eval Results:", eval_results)
        
        # Save the model
        local_path = f"../models_ccu/{MODEL.split('/')[0]}_sampling_{strategie.replace('/','_')}_{topic}_model_{FEATURES}"
        trainer.save_model(local_path)
        tokenizer.save_pretrained(local_path)
        
        # Clear GPU memory to avoid memory errors
        del model, tokenizer, tokenized_datasets, trainer
        torch.cuda.empty_cache()

Training on cannabis dataset using random strategy


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at deepset/gelectra-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 507/507 [00:00<00:00, 3174.25 examples/s]
/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
100,0.309300,0.008089,1.000000


/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Eval Results: {'eval_loss': 0.002923979191109538, 'eval_accuracy': 1.0, 'eval_runtime': 23.7244, 'eval_samples_per_second': 21.37, 'eval_steps_per_second': 2.698, 'epoch': 3.0}
Training on cannabis dataset using stratified strategy


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at deepset/gelectra-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 507/507 [00:00<00:00, 2461.75 examples/s]
/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


KeyboardInterrupt: 

In [ ]:
import json

# Define the file path to save the dictionary
file_path = f"training_results_sampling_{FEATURES}.json"

# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(training_results, file)